In [ ]:
from typing import List, Dict
# contient le tokenizer et le modèle de sentiment
from transformers import pipeline, logging
from loguru import logger
# sert à faire les calculs du modèle
import torch
from elasticsearch import Elasticsearch
import pandas as pd

In [3]:
# Désactive les messages info de Transformers
logging.set_verbosity_error()

In [4]:
def convert_stars_to_sentiment(label: str) -> str:
    """
    Convertit une prédiction du modèle exprimée en étoiles
    (1 à 5 étoiles) en un sentiment textuel simplifié.

    Les correspondances sont les suivantes :
    - 1 ou 2 étoiles → NEGATIF
    - 3 étoiles      → NEUTRE
    - 4 ou 5 étoiles → POSITIF

    Cette fonction permet de normaliser la sortie du modèle
    de sentiment en trois catégories exploitables.

    Parameters
    ----------
    label : str
        Label retourné par le modèle Hugging Face
        (ex. "1 star", "3 stars", "5 stars").

    Returns
    -------
    str
        Sentiment normalisé : "NEGATIF", "NEUTRE" ou "POSITIF".

    Raises
    ------
    ValueError
        Si le label fourni ne correspond pas à une valeur attendue.

    """
    label = label.upper()

    if label in ["1 STAR", "2 STARS"]:
        return "NEGATIF"
    elif label == "3 STARS":
        return "NEUTRE"
    elif label in ["4 STARS", "5 STARS"]:
        return "POSITIF"
    else:
        raise ValueError(f"Label inattendu : {label}")

In [6]:
def predict_sentiment_batch(texts: List[str]) -> List[Dict[str, str]]:
    """
    Prédit le sentiment d'une liste d'avis utilisateurs à partir d'un modèle
    de traitement du langage naturel (NLP).

    Le modèle utilisé est :
    - cmarkea/distilcamembert-base-sentiment (Hugging Face)

    Le modèle est entraîné à prédire une note de 1 à 5 étoiles à partir d'un texte.
    Une étape de normalisation convertit ensuite ces notes en trois catégories :
    - NEGATIF (1-2 étoiles)
    - NEUTRE  (3 étoiles)
    - POSITIF (4-5 étoiles)

    Le modèle est chargé via `pipeline()` avec un tokenizer lent (`use_fast=False`)
    afin d'assurer la stabilité et la compatibilité sous Windows.

    Parameters
    ----------
    texts : List[str]
        Liste de textes correspondant aux avis utilisateurs
        (ex. récupérés depuis Elasticsearch).

    Returns
    -------
    List[Dict[str, str]]
        Liste de dictionnaires contenant pour chaque avis :
        - text_clean : texte nettoyé
        - sentiment  : sentiment prédit (POSITIF, NEUTRE ou NEGATIF)
    """

    if not texts:
        raise ValueError("La liste de textes est vide.")

    # Chargement du modèle (une seule fois)
    model = pipeline(
        task="sentiment-analysis",
        model="cmarkea/distilcamembert-base-sentiment",
        tokenizer="cmarkea/distilcamembert-base-sentiment",
        use_fast=False,
        truncation=True,
    )

    results = []

    for text in texts:

        prediction = model(text, max_length=512)[0]
        sentiment = convert_stars_to_sentiment(prediction["label"])

        results.append(
            {
                "text_clean": text,
                "sentiment": sentiment
            }
        )

    return results

In [11]:
try:

    es = Elasticsearch("http://localhost:9200")
    if not es.ping():
        raise ConnectionError("Impossible de se connecter à Elasticsearch.")
except Exception as error:
    logger.exception(
            f"Impossible de se connecter à Elasticsearch: {error}")
    raise

try:

    response = es.search(
        index="reviews",
        query={
            "exists": {
                "field": "user_review"
            }
        },
        size=200
    )

    reviews = []
    for hit in response["hits"]["hits"]:
        reviews.append(hit["_source"]["user_review"])
except Exception as error:
    logger.exception(f"Erreur lors de la récupération des avis: {error}")

In [ ]:
predictions = predict_sentiment_batch(reviews)

datas = []
for review, prediction in zip(reviews, predictions):
    datas.append({
        "user_review": review,
        "sentiment_prédit": prediction["sentiment"],
    })

df = pd.DataFrame(datas)
# je récupère les 20 dernières lignes du dataframe afin prendre les avis les plus récents
df.tail(20)

,user_review,sentiment_prédit
180,Produit parfait correspondant à la description...,POSITIF
181,Colis parfait et contente des produits,POSITIF
182,Toujours parfaitement satisfaite de Showroom. ...,POSITIF
183,On trouve toujours des pépites. La livraison e...,POSITIF
184,Produits conformes et livraison dans les temps...,POSITIF
185,La bague est conforme à ce que j'attendais,POSITIF
186,Colis recu ouvert/ AI signale le prob jamais e...,NEGATIF
187,vente intéressante mais l'envoi est trop long,NEUTRE
188,tres bon acceuil,POSITIF
189,Le colis est arrivé et a l'ouverture du colis ...,NEGATIF
